### Modelamiento ML
El modelamiento en el contexto del aprendizaje automático (machine learning) se refiere al proceso de crear y entrenar un modelo predictivo utilizando algoritmos y datos. Este proceso implica seleccionar el tipo de modelo adecuado para el problema en cuestión, preparar y limpiar los datos, elegir las características relevantes, entrenar el modelo con los datos de entrenamiento y evaluar su rendimiento con datos de prueba o validación.



### Importamos librerías

Comienza instalando solo las librerías que necesitas para empezar tu proyecto.

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity

### La lectura de datos 

Lectura del archivo **modelo_games.parquet**

In [2]:
modelo_games = pq.read_table("../0-DATA/modelo_games.parquet").to_pandas()
modelo_games.head(5)

,item_id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,643980,Ironbound,Free to Play
2,670290,Real Pool 3D - Poolians,Casual
3,767400,弹炸人2222,Action
4,772540,Battle Royale Trainer,Action


In [3]:
# Creamos variables dummy para la columna "genres" en el DataFrame "modelo_games"
# Esto convierte las categorías en variables binarias (0 o 1)
# Cada género se convertirá en una nueva columna con valores binarios indicando la presencia o ausencia del género para cada entrada
modelo_item = pd.get_dummies(modelo_games, columns=["genres"], prefix="")

# Agrupamos por "item_id" y "app_name" y sumamos los valores binarios de los géneros para cada juego
# Esto nos dará una fila por juego con la suma de las variables dummy de género para ese juego
modelo_item = modelo_item.groupby(["item_id","app_name"]).sum().reset_index()

# Visualizamos las primeras 4 filas del DataFrame resultante
modelo_item.head(4)


,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Calculamos la similitud coseno entre las características de los juegos utilizando la función cosine_similarity de scikit-learn
# modelo_item.iloc[:,3:] selecciona todas las filas y todas las columnas a partir de la cuarta columna (índice de columna 3) hasta el final,
# ya que las primeras tres columnas probablemente contienen identificadores o nombres de juegos, no características
similitudes = cosine_similarity(modelo_item.iloc[:, 3:])

# El resultado es una matriz de similitud donde cada entrada (i, j) representa la similitud coseno entre el juego i y el juego j
# La diagonal de la matriz contiene la similitud de cada juego consigo mismo, que es 1 por definición de similitud coseno
# El valor de similitud está entre -1 y 1, donde 1 significa que los juegos son idénticos y -1 significa que son completamente diferentes

In [5]:
similitudes.shape

(27461, 27461)

In [6]:
print(similitudes)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


Dividimos el dataframe por la mitad a fin de lograr montar el modelo en render



In [7]:
# Revisamos la cantidad de filas
cant_filas= len(modelo_item)

# Calculamos la mitad
mitad_filas= cant_filas // 10
 
# Seleccionamos la mitad superior
modelo_render= modelo_item.iloc[:mitad_filas]

In [9]:
modelo_render.shape

(2746, 24)

Dataframe para Render FastApi

In [11]:
tabla = pa.Table.from_pandas(modelo_render) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"../0-DATA/modelo_render.parquet") # guardamos la tabla en formato parquet

In [12]:
modelo_render.to_csv("../0-DATA/modelo_render.csv", index=False, encoding="utf-8") 

Función para recomendación de juegos en Render FastApi

In [13]:
def recomendacion_juego(id: int):
    
    # Verificamos si el juego con game_id existe en df_games
    game = modelo_render[modelo_render['item_id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtenemos el índice del juego dado
    idx = game.index[0]

    # Tomamos una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Definimos el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_render.sample(n=sample_size, random_state=42)  # Ajustamos la semilla aleatoria según sea necesario

    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_render.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [14]:
recomendacion_juego(3310)

{'similar_games': ['Hidden Expedition: Titanic',
  'Dwarfs - F2P Difficulty Pack',
  'Rocksmith - Guitarcade - Time Saver Pack',
  'Virtual Villagers: A New Home',
  'The Otherside: Realm of Eons']}